# Quadratic problems (advanced)

This example shows an advanced quadratic problems usage, e.g., the {class}`~moscot.problems.time.LineageProblem`, the {class}`~moscot.problems.spatio_temporal.SpatioTemporalProblem`, the {class}`~moscot.problems.space.MappingProblem`, the {class}`~moscot.problems.space.AlignmentProblem`, and the {class}`~moscot.problems.generic.GWProblem`.

:::{seealso}
- See {doc}`300_quad_problems_basic` for an introduction on how to solve quadratic problems.
- See {doc}`100_linear_problems_basic` for an introduction on how to solve linear problems.
- See {doc}`200_linear_problems_advanced` for an advanced example on how to solve linear problems.
:::

## Imports and data loading

In [1]:
from moscot import datasets
from moscot.problems.generic import GWProblem

import scanpy as sc

Simulate data using {func}`~moscot.datasets.simulate_data`.

In [2]:
adata = datasets.simulate_data(n_distributions=2, key="batch", quad_term="spatial")
sc.pp.pca(adata)
adata

AnnData object with n_obs × n_vars = 40 × 60
    obs: 'batch', 'celltype'
    uns: 'pca'
    obsm: 'spatial', 'X_pca'
    varm: 'PCs'

In [3]:
gwp = GWProblem(adata)
gwp = gwp.prepare(
    key="batch",
    GW_x={"attr": "obsm", "key": "spatial"},
    GW_y={"attr": "obsm", "key": "spatial"},
)
gwp

INFO     Computing pca with `n_comps=30` for `xy` using `adata.X`                                                  


GWProblem[('0', '1')]

## Threshold

The `threshold` parameter defines the convergence criterion. In the balanced
setting the `threshold` denotes the deviation between prior and posterior
marginals, while in the unbalanced setting the `threshold` corresponds to
a Cauchy sequence stopping criterion.

## Initializers

Different Initializers can help to improve convergence. For the full-rank
case only the default initializer exists, hence the `initializer` argument
must be set to {obj}`None`.

For low-rank problems the same initializers as for the linear low-rank solvers
are available, and `initializer_kwargs` can be passed the same way, see {doc}`200_linear_problems_advanced` for more information.

## Number of iterations

To solve a quadratic optimal transport problem, a consecutively-updated linearized
problem is solved. Here, `min_iterations` denotes a lower bound and `max_iterations` an upper bound on the number of outer iterations. If `max_iterations` is too low, the solver might not converge.

In [4]:
gwp = gwp.solve(alpha=0.5, epsilon=1e-1, min_iterations=0, max_iterations=1)

INFO     Solving problem OTProblem[stage='prepared', shape=(20, 20)].                                              
WARNING  Solver did not converge                                                                                   


## Linear solver keyword arguments

As mentioned above, each outer loop step of the Gromov-Wasserstein algorithm
consists of solving a linear problem. Arguments for the linear solver can
be specified via `linear_solver_kwargs`, keyword arguments for
{class}`~ott.solvers.linear.sinkhorn.Sinkhorn` in the full-rank case or keyword arguments
for {class}`~ott.solvers.linear.sinkhorn_lr.LRSinkhorn`, respectively. This way, we can
also set the minimum and maximum number of iterations for the linear solver:

In [5]:
ls_kwargs = {"min_iterations": 10, "max_iterations": 1000, "threshold": 0.01}
gwp = gwp.solve(
    alpha=0.5,
    epsilon=1e-1,
    threshold=0.1,
    min_iterations=2,
    max_iterations=20,
    linear_solver_kwargs=ls_kwargs,
)

INFO     Solving problem OTProblem[stage='solved', shape=(20, 20)].                                                
WARNING  Solver did not converge                                                                                   


## Low-rank hyperparameters

The parameters `gamma` and `gamma_rescale` are the same as in the linear case, see example {doc}`200_linear_problems_advanced`.

<!--- #It remains to consider ``ranks`` and ``tolerances``. --->

## Keyword arguments and implementation details

Whenever the `solve` method of a quadratic problem is called, a backend-specific quadratic solver is instantiated. Currently, {mod}`ott` is supported, its corresponding quadratic solvers is {class}`~ott.solvers.quadratic.gromov_wasserstein.GromovWasserstein`, handling both the full-rank and the low-rank case. {mod}`moscot` wraps this class in {class}`~moscot.backends.ott.GWSolver`, handling both the purely quadratic and the fused quadratic problem.